# Spherical harmonics

## References

* [Spherical Harmonics for Beginners](https://dickyjim.wordpress.com/2013/09/04/spherical-harmonics-for-beginners/)
* [Spherical Harmonic Lighting: The Gritty Details](references/spherical-harmonic-lighting.pdf)
* [Spherical Harmonics in Actual Games](https://tomforsyth1000.github.io/papers/papers.html)
* [Stupid Spherical Harmonics (SH) Tricks](http://www.ppsloan.org/publications/StupidSH36.pdf)
* [An Efficient Representation for Irradiance Environment Maps](http://cseweb.ucsd.edu/~ravir/papers/envmap/envmap.pdf)
* [On the relationship between radiance andirradiance:  determiningthe illumination fromimages of a convex Lambertian object](http://cseweb.ucsd.edu/~ravir/papers/invlamb/josa.pdf)
* [GameDev.net - Spherical Harmonics Cubemap](https://www.gamedev.net/forums/topic/671562-spherical-harmonics-cubemap/)
* [HLSL-Spherical-Harmonics](https://github.com/sebh/HLSL-Spherical-Harmonics)  
* [Dive in SH buffer idea](https://seblagarde.wordpress.com/2011/10/09/dive-in-sh-buffer-idea/)

## Utility code snippets

In [ ]:
import numpy as np
import sph_harm as sh

def spherical_integral(integrand, numsamples):
    dtheta = np.pi / (numsamples - 1)
    dphi = 2*np.pi / (2*numsamples - 1)

    theta = np.arange(0, np.pi, dtheta)
    phi = np.arange(0, np.pi*2, dphi)
    theta, phi = np.meshgrid(theta, phi)
    vals = integrand(theta, phi) * np.sin(theta)

    return vals.sum() * dtheta * dphi

def spherical_dir(theta, phi):
    x = np.sin(theta) * np.cos(phi)
    y = np.sin(theta) * np.sin(phi)
    z = np.cos(theta)
    return x, y, z

def spherical_coord(x, y, z):
    norm = np.sqrt(x**2 + y**2 + z**2)
    theta = np.arccos(z/norm)
    phi = np.arctan2(y, x)
    return theta, phi

def meshgrid_spherical_coord(numsamples):
    theta = np.linspace(0, np.pi, num=numsamples)
    phi = np.linspace(0, 2*np.pi, num=numsamples*2)
    theta, phi = np.meshgrid(theta, phi)
    return theta, phi

def sph_harm_xyz(m, l, x, y, z):
    theta, phi = spherical_coord(x, y, z)
    return sh.sph_harm(m, l, theta, phi)

def print_sh_coeffs(band, coeffs):
    for l in np.arange(band):
        linestr = ""
        for m in np.arange(-l, l+1):
            i = sh_idx(m, l)
            linestr += str(coeffs[i]) + " "
        print(linestr)

## SH coefficient visualization

In [ ]:
import numpy as np
import sph_harm as sh
import ipyvolume as ipv
from matplotlib import cm, colors


# color map for SH coefficient visualization
N = 256
vals = np.zeros((N, 4))
vals[:N//2, 0] = np.linspace(1, 0, N//2)
vals[N//2:, 1] = np.linspace(0, 1, N//2)
vals[:, 3] = np.ones(N)
sh_colormap = colors.ListedColormap(vals)

theta, phi = meshgrid_spherical_coord(128)
x, y, z = spherical_dir(theta, phi)

sh_band = 5

ipv.figure(lighting=False)
ipv.xyzlim(-sh_band*2, sh_band*2)
ipv.ylim(-sh_band*3, sh_band)

for l in np.arange(sh_band):
    for m in np.arange(-l, l+1):
        sh_vals = sh.sph_harm(m, l, theta, phi).real
#         sh_vals = sph_harm_xyz(m, l, x, y, z).real
        abs_sh = np.abs(sh_vals)
        sh_min, sh_max = sh_vals.min(), sh_vals.max()
        sh_max += 0.000001
        sh_gradient = (sh_vals - sh_min) / (sh_max - sh_min)

        cmap = sh_colormap(sh_gradient)
        if l == 0:
            # hack the cmap for first order cause it's a constant so that gradient is zero
            cmap = "green"

        coord_offset = np.array([2*m, -2*l, 0])
        ipv.plot_mesh(x*abs_sh+coord_offset[0], z*abs_sh+coord_offset[1], y*abs_sh+coord_offset[2], wireframe=False, color=cmap)
        sphere_scale = 0.2
        sphere_offset = coord_offset+np.array([0.5, 0.5, 0])
        ipv.plot_mesh(x*sphere_scale+sphere_offset[0], z*sphere_scale+sphere_offset[1], y*sphere_scale+sphere_offset[2], wireframe=False, color=cmap)
ipv.show()

## SH projection example

Define a function that we want to project into SH basis

In [ ]:
import numpy as np
import sph_harm as sh
import ipyvolume as ipv
from matplotlib import cm

def func(theta, phi):
    l = np.maximum(0, 5*np.cos(theta)-4) + np.maximum(0, -4*np.sin(theta-np.pi)*np.cos(phi-2.5+np.pi)-3)
    return l

Visualize it

In [ ]:
theta, phi = meshgrid_spherical_coord(128)
x, y, z = spherical_dir(theta, phi)

vals = func(theta, phi)

ipv.figure(lighting=False)
ipv.xyzlim(-1, 1)
ipv.plot_mesh(x, z, y, wireframe=False, color=cm.coolwarm(vals))
ipv.show()

Another way to visualize it

In [ ]:
ipv.figure(lighting=False)
ipv.xyzlim(-1, 1)
ipv.plot_mesh(x*vals, z*vals, y*vals, wireframe=False, color=cm.coolwarm(vals))
ipv.show()

Now define SH projection and reconstruction functions

In [ ]:
def sh_projection(spherical_func, band, numsamples):
    sh_coeffs = np.ndarray(band**2)
    sh_coeffs.fill(0)

    for l in range(band):
        for m in range(-l, l + 1):
            def integrand(theta, phi):
                return spherical_func(theta, phi) * sh.sph_harm(m, l, theta, phi).real
#                 x = np.sin(theta) * np.cos(phi)
#                 y = np.sin(theta) * np.sin(phi)
#                 z = np.cos(theta)
#                 return spherical_func(theta, phi) * sph_harm_xyz(m, l, x, y, z).real

            i = sh.sh_idx(m, l)
            sh_coeffs[i] = spherical_integral(integrand, numsamples)
    return sh_coeffs

# zonal harmonics
def zh_projection(spherical_func, band, numsamples):
    sh_coeffs = np.ndarray(band**2)
    sh_coeffs.fill(0)

    for l in range(band):
        m = 0

        def integrand(theta, phi):
            return spherical_func(theta, phi) * sh.sph_harm(m, l, theta, phi).real

        i = sh.sh_idx(m, l)
        sh_coeffs[i] = spherical_integral(integrand, numsamples)
    return sh_coeffs

def zh_coeffs_replicate(band, zh_coeffs):
    sh_coeffs = zh_coeffs
    for l in range(band):
        for m in range(-l, l + 1):
            if m != 0:
                base = sh.sh_idx(0, l)
                i = sh.sh_idx(m, l)
                sh_coeffs[i] = zh_coeffs[base]
    return sh_coeffs

def sh_reconstruction(theta, phi, band, sh_coeffs):
    vals = np.zeros_like(theta)
    for l in range(band):
        for m in range(-l, l + 1):
            sh_val = sh.sph_harm(m, l, theta, phi).real
            i = sh.sh_idx(m, l)
            coeff = sh_coeffs[i]
            vals = vals + sh_val * coeff
    return vals

Visualize the difference between original function and reconstructed function

In [ ]:
theta, phi = meshgrid_spherical_coord(128)
x, y, z = spherical_dir(theta, phi)

sh_band = 6
coeff = sh_projection(func, sh_band, 256)
diff = sh_reconstruction(theta, phi, sh_band, coeff) - func(theta, phi)

ipv.figure(lighting=False)
ipv.xyzlim(-1, 1)
ipv.plot_mesh(x*diff, z*diff, y*diff, wireframe=False, color=cm.coolwarm(diff))
ipv.show()

## Zonal harmonics rotation

In [ ]:
import numpy as np
import ipyvolume as ipv
from matplotlib import cm

# define a radial symmetry function
def func(theta, phi):
    return np.maximum(0, np.cos(theta))

In [ ]:
# define a helper function that apply a rotation to a spherical function
def rotate_spherical_func(func, rotation, x, y, z):
    rot_matrix = rotation.as_dcm().transpose()
    x_p = rot_matrix[0][0]*x+rot_matrix[0][1]*y+rot_matrix[0][2]*z
    y_p = rot_matrix[1][0]*x+rot_matrix[1][1]*y+rot_matrix[1][2]*z
    z_p = rot_matrix[2][0]*x+rot_matrix[2][1]*y+rot_matrix[2][2]*z
    theta_p = np.arccos(z_p)
    phi_p = np.arctan2(y_p, x_p)
    return func(theta_p, phi_p)

In [ ]:
from scipy.spatial.transform import Rotation as R

theta, phi = meshgrid_spherical_coord(128)
x, y, z = spherical_dir(theta, phi)

# define a rotation
rotation = R.from_euler('y', 90, degrees=True)

vals = rotate_spherical_func(func, rotation, x, y, z)

ipv.figure(lighting=False)
ipv.xyzlim(-1, 1)
ipv.plot_mesh(x, y, z, wireframe=False, color=cm.coolwarm(vals))
ipv.show()

In [ ]:
vec = np.array([0, 0, 1])
vec_p = rotation.apply(vec)

sh_bands = 6
sh_coeffs = zh_projection(func, sh_bands, 128)
sh_coeffs = zh_coeffs_replicate(sh_bands, sh_coeffs)
for l in np.arange(sh_bands):
    for m in np.arange(-l, l+1):
        i = sh.sh_idx(m, l)
        # rotate zonal harmonics
        sh_coeffs[i] = np.sqrt(4*np.pi/(2*l+1))*sph_harm_xyz(m, l, vec_p[0], vec_p[1], vec_p[2])*sh_coeffs[i]

diff = sh_reconstruction(theta, phi, sh_bands, sh_coeffs) - vals

ipv.figure(lighting=False)
ipv.xyzlim(-1, 1)
ipv.plot_mesh(x, y, z, wireframe=False, color=cm.coolwarm(diff))
ipv.show()